In [35]:
import random
import numpy as np
import torch
from torchinfo import summary
import torch.nn as nn


def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [9]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import flwr
from flwr.client import Client, ClientApp, NumPyClient
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.server.strategy import FedAvg, FedAdagrad
from flwr.simulation import run_simulation
from flwr_datasets import FederatedDataset
from flwr.common import ndarrays_to_parameters, NDArrays, Scalar, Context
from flwr.common import Metrics, Context


from flwr_datasets import FederatedDataset
from flwr_datasets.partitioner import IidPartitioner

from flwr.client.mod import parameters_size_mod

from dmf import *

In [10]:
DEVICE = torch.device("cuda") 
NUM_PARTITIONS = 5
BATCH_SIZE = 256

print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")

dataset = "ashraq/movielens_ratings" 
partitioner = IidPartitioner(num_partitions=NUM_PARTITIONS)           
fds = FederatedDataset(dataset=dataset,
                    partitioners={"train": partitioner})

Training on cuda
Flower 1.10.0 / PyTorch 2.2.1+cu121


/home/haya/miniconda/envs/fedrec/lib/python3.9/site-packages/flwr_datasets/utils.py:109: UserWarning: The currently tested dataset are ['mnist', 'ylecun/mnist', 'cifar10', 'uoft-cs/cifar10', 'fashion_mnist', 'zalando-datasets/fashion_mnist', 'sasha/dog-food', 'zh-plus/tiny-imagenet', 'scikit-learn/adult-census-income', 'cifar100', 'uoft-cs/cifar100', 'svhn', 'ufldl-stanford/svhn', 'sentiment140', 'stanfordnlp/sentiment140', 'speech_commands', 'LIUM/tedlium', 'flwrlabs/femnist', 'flwrlabs/ucf101', 'flwrlabs/ambient-acoustic-context', 'jlh/uci-mushrooms', 'Mike0307/MNIST-M', 'flwrlabs/usps', 'scikit-learn/iris', 'flwrlabs/pacs', 'flwrlabs/cinic10', 'flwrlabs/caltech101', 'flwrlabs/office-home', 'flwrlabs/fed-isic2019']. Given: ashraq/movielens_ratings.
  warnings.warn(


### Global Mapping

In [11]:
########################################
# Process the Federated Dataset & Global Mapping without interaction matrix
########################################

def compute_global_mapping(fds):
    """
    Compute global mapping for user and item IDs from the full dataset.
    """


    # Load the full splits from the FederatedDataset (adjust split names as needed)
    global_train_df = fds.load_split("train").to_pandas()[["user_id", "movie_id", "rating"]]    # Full training data across all clients
    global_test_df = fds.load_split("validation").to_pandas()[["user_id", "movie_id", "rating"]]    # Full test set

    # Exclude cold cases from valid/test based solely on training data.
    train_users = set(global_train_df['user_id'].unique())
    train_movies = set(global_train_df['movie_id'].unique())

    global_test_df = global_test_df[
        global_test_df['user_id'].isin(train_users) &
        global_test_df['movie_id'].isin(train_movies)
    ]

    # Build the union of all user and movie IDs across splits.
    all_users = set(global_train_df['user_id']).union(global_test_df['user_id'])
    all_movies = set(global_train_df['movie_id']).union(global_test_df['movie_id'])

    # Create mapping dictionaries (zero-based, contiguous indices)
    user_id_map = {user: idx for idx, user in enumerate(sorted(all_users))}
    movie_id_map = {movie: idx for idx, movie in enumerate(sorted(all_movies))}

    num_users = len(user_id_map)
    num_movies = len(movie_id_map)
    print("Global Number of Users:", num_users)
    print("Global Number of Movies:", num_movies)

    return user_id_map, movie_id_map,

    
# Precompute the global mappings and interaction matrix once.
global_user_id_map, global_movie_id_map = compute_global_mapping(fds)


Global Number of Users: 43584
Global Number of Movies: 15276


### Load Datastes

In [18]:
########################################
# DataLoader Setup Using Global Mappings
########################################

def load_datasets(partition_id: int, batch_size: int, global_user_id_map, global_movie_id_map):
    
    # Load the partition assigned to this client.
    partition = fds.load_partition(partition_id)
    partition_train_test = partition.train_test_split(test_size=0.2, seed=42)


    train = partition_train_test["train"].to_pandas()[["user_id", "movie_id", "rating"]] 
    valid = partition_train_test["test"].to_pandas()[["user_id", "movie_id", "rating"]] 
    
    # Filter out rows with IDs that are not in the global mapping (if any).
    valid = valid[
        valid['user_id'].isin(global_user_id_map) & valid['movie_id'].isin(global_movie_id_map)
    ]
    
    # Create DMFDataset instances using the global mapping.
    train_dataset = DMFDataset(train, global_user_id_map, global_movie_id_map)
    valid_dataset = DMFDataset(valid, global_user_id_map, global_movie_id_map)

    
    # Build DataLoaders.
    trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    test = fds.load_split("validation").to_pandas()[["user_id", "movie_id", "rating"]] 

    test = test[
        test['user_id'].isin(global_user_id_map) & test['movie_id'].isin(global_movie_id_map)
    ]

    test_dataset = DMFDataset(test, global_user_id_map, global_movie_id_map)
    testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    
    return trainloader, valloader, testloader

### Proposed Model architecture

In [37]:
########################################
# The model
########################################

class MLPLayers(nn.Module):
    def __init__(self, sizes, dropout=0.3, activation="leaky_relu", bn=False, init_method="norm", last_activation=True):
        super(MLPLayers, self).__init__()
        layers = []
        for i in range(len(sizes) - 1):
            layers.append(nn.Linear(sizes[i], sizes[i+1]))
            if bn:
                layers.append(nn.BatchNorm1d(sizes[i+1]))
            if activation == "leaky_relu":
                layers.append(nn.LeakyReLU())
            else:
                layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
        if not last_activation:
            layers = layers[:-2]
        self.mlp = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.mlp(x)

class DMFFederated(nn.Module):
    """
    Modified DMF model that uses learnable embeddings instead of a precomputed global interaction matrix.
    """
    def __init__(self, num_users, num_items,
                 user_embedding_size=32,
                 item_embedding_size=32,
                 user_hidden_sizes=[64, 32],
                 item_hidden_sizes=[64, 32],
                 dropout=0.3,
                 activation="leaky_relu",
                 bn=False,
                 init_method="norm"):
        super(DMFFederated, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        
        self.user_embedding = nn.Embedding(num_users, user_embedding_size)
        self.item_embedding = nn.Embedding(num_items, item_embedding_size)
        
        self.user_fc_layers = MLPLayers(
            [user_embedding_size] + user_hidden_sizes,
            dropout=dropout,
            activation=activation,
            bn=bn,
            init_method=init_method,
            last_activation=True
        )
        self.item_fc_layers = MLPLayers(
            [item_embedding_size] + item_hidden_sizes,
            dropout=dropout,
            activation=activation,
            bn=bn,
            init_method=init_method,
            last_activation=True
        )
        
        self.loss_fn = nn.HuberLoss(delta=0.5)
        self._init_weights()
    
    def _init_weights(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.normal_(module.weight, 0, 0.01)
                if module.bias is not None:
                    module.bias.data.fill_(0.0)
            elif isinstance(module, nn.Embedding):
                nn.init.normal_(module.weight, 0, 0.01)
    
    def forward(self, user_indices, item_indices):
        user_emb = self.user_embedding(user_indices)
        item_emb = self.item_embedding(item_indices)
        user_features = self.user_fc_layers(user_emb)
        item_features = self.item_fc_layers(item_emb)
        prediction = torch.mul(user_features, item_features).sum(dim=1)
        return prediction
    
    def calculate_loss(self, batch):
        user = batch['user_id']
        item = batch['movie_id']
        rating = batch['rating']
        preds = self.forward(user, item)
        loss = self.loss_fn(preds, rating)
        return loss
    
    def predict(self, batch):
        return self.forward(batch['user_id'], batch['movie_id'])

model = DMFFederated(
            num_users=len(global_user_id_map),
            num_items=len(global_movie_id_map),
            user_embedding_size=32,
            item_embedding_size=32,
            user_hidden_sizes=[64, 32],
            item_hidden_sizes=[64, 32],
            dropout=0.3,
            activation="leaky_relu",
            bn=False,
            init_method="norm"
            )




In [39]:
########################################
# Print model
########################################
print(model)
print("-"*25)

class WrappedModel(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.base = base_model

    def forward(self, user_ids, item_ids):
        return self.base(user_ids, item_ids)

wrapped = WrappedModel(model)

summary(
    wrapped,
    input_size=[(16,), (16,)],          
    dtypes=[torch.long, torch.long]      
)

DMFFederated(
  (user_embedding): Embedding(43584, 32)
  (item_embedding): Embedding(15276, 32)
  (user_fc_layers): MLPLayers(
    (mlp): Sequential(
      (0): Linear(in_features=32, out_features=64, bias=True)
      (1): LeakyReLU(negative_slope=0.01)
      (2): Dropout(p=0.3, inplace=False)
      (3): Linear(in_features=64, out_features=32, bias=True)
      (4): LeakyReLU(negative_slope=0.01)
      (5): Dropout(p=0.3, inplace=False)
    )
  )
  (item_fc_layers): MLPLayers(
    (mlp): Sequential(
      (0): Linear(in_features=32, out_features=64, bias=True)
      (1): LeakyReLU(negative_slope=0.01)
      (2): Dropout(p=0.3, inplace=False)
      (3): Linear(in_features=64, out_features=32, bias=True)
      (4): LeakyReLU(negative_slope=0.01)
      (5): Dropout(p=0.3, inplace=False)
    )
  )
  (loss_fn): HuberLoss()
)
-------------------------


Layer (type:depth-idx)                   Output Shape              Param #
WrappedModel                             [16]                      --
├─DMFFederated: 1-1                      [16]                      --
│    └─Embedding: 2-1                    [16, 32]                  1,394,688
│    └─Embedding: 2-2                    [16, 32]                  488,832
│    └─MLPLayers: 2-3                    [16, 32]                  --
│    │    └─Sequential: 3-1              [16, 32]                  4,192
│    └─MLPLayers: 2-4                    [16, 32]                  --
│    │    └─Sequential: 3-2              [16, 32]                  4,192
Total params: 1,891,904
Trainable params: 1,891,904
Non-trainable params: 0
Total mult-adds (M): 30.27
Input size (MB): 0.00
Forward/backward pass size (MB): 0.03
Params size (MB): 7.57
Estimated Total Size (MB): 7.60

In [7]:
########################################
# Set and get parameters
########################################

from collections import OrderedDict
from typing import Dict, List, Optional, Tuple, Union


def set_parameters(model, parameters: List[np.ndarray]):
    """
    Sets the parameters of the model using a list of NumPy arrays.
    
    Args:
        model (torch.nn.Module): The model.
        parameters (List[np.ndarray]): The model parameters as a list of NumPy arrays.
    """
    params_dict = zip(model.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.from_numpy(v) for k, v in params_dict})
    model.load_state_dict(state_dict, strict=True)


def get_parameters(model) -> List[np.ndarray]:
    """
    Retrieves the model parameters as a list of NumPy arrays.

    Args:
        model (torch.nn.Module): The model.

    Returns:
        List[np.ndarray]: The model parameters as a list of NumPy arrays.
    """
    return [val.cpu().numpy() for _, val in model.state_dict().items()]


In [8]:
########################################
# Training and Evaluation Functions
########################################

def train(model, trainloader, epochs: int, lr=0.0001, weight_decay=1e-4, device="cpu"):
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    model.train()
    for epoch in range(epochs):
        total_loss = 0.0
        total_samples = 0
        for batch in trainloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            optimizer.zero_grad()
            loss = model.calculate_loss(batch)  
            loss.backward()
            optimizer.step()
            bs = len(batch['rating'])
            total_loss += loss.item() * bs
            total_samples += bs
        avg_loss = total_loss / total_samples if total_samples > 0 else 0.0
        print(f"Epoch {epoch+1}/{epochs}, Training Loss: {avg_loss:.4f}")

        

def test(model, dataloader, device="cpu"):
    model.eval()
    total_loss = 0.0
    total_samples = 0
    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            loss = model.calculate_loss(batch)
            bs = len(batch['rating'])
            total_loss += loss.item() * bs
            total_samples += bs
    avg_loss = total_loss / total_samples if total_samples > 0 else 0.0
    print(f"Evaluation Loss: {avg_loss:.4f}")
    return avg_loss


In [ ]:
########################################
# Inilialize the model 
########################################

model = DMFFederated(
            num_users=len(global_user_id_map),
            num_items=len(global_movie_id_map),
            user_embedding_size=32,
            item_embedding_size=32,
            user_hidden_sizes=[64, 32],
            item_hidden_sizes=[64, 32],
            dropout=0.3,
            activation="leaky_relu",
            bn=False,
            init_method="norm"
            )

params = get_parameters(model)

### Client Side

In [9]:
########################################
# Flower Client Definition and Client Function
########################################
torch.cuda.empty_cache()
device = DEVICE 
num_partitions = NUM_PARTITIONS
batch_size = BATCH_SIZE
num_epochs = 1
lr = 0.0001
weight_decay = 1e-4


class FlowerClient(NumPyClient):
    def __init__(self, partition_id, model, trainloader, valloader):
        self.partition_id = partition_id
        self.model = model
        self.trainloader = trainloader
        self.valloader = valloader
        

    def get_parameters(self, config):
        params = get_parameters(self.model)
        return params
    
   
    def fit(self, parameters, config):
        print(f"[Client {self.partition_id}] fit, config: {config}")
        set_parameters(self.model, parameters)
        train(self.model, self.trainloader, epochs=num_epochs, lr=lr, weight_decay=weight_decay, device=device)
        return get_parameters(self.model), len(self.trainloader), {}
    


    def evaluate(self, parameters, config):
        print(f"[Client {self.partition_id}] evaluate, config: {config}")
        set_parameters(self.model, parameters)
        loss = test(self.model, self.valloader, device=device)
        return float(loss), len(self.valloader), {"loss": float(loss)}
    

def client_fn(context: Context) -> Client:
    partition_id = context.node_config["partition-id"]

    model = DMFFederated(
            num_users=len(global_user_id_map),
            num_items=len(global_movie_id_map),
            user_embedding_size=32,
            item_embedding_size=32,
            user_hidden_sizes=[64, 32],
            item_hidden_sizes=[64, 32],
            dropout=0.3,
            activation="leaky_relu",
            bn=False,
            init_method="norm"
            ).to(device)

    
    trainloader, valloader, _ = load_datasets(
        partition_id, batch_size,  global_user_id_map, global_movie_id_map
    )

    return FlowerClient(partition_id, model, trainloader, valloader).to_client()


client_app = ClientApp(client_fn=client_fn)
client_app 

### Server Side

In [11]:
def evaluate_DMFRegressor(model, test_loader, device, batch_size=64):
    """
    Evaluate the regression model on the test set.
    
    Args:
        model: The DMFModel (or any regression model) instance that predicts continuous ratings.
        test_set: A PyTorch Dataset (e.g., DMFDataset) for testing.
        device: The torch.device (e.g., "cuda" or "cpu").
        batch_size (int): Batch size for evaluation.
    
    Returns:
        average_loss: The average loss over the test set.
        mae: Mean Absolute Error.
        rmse: Root Mean Squared Error.
        r2: R^2 score (coefficient of determination).
    """
    
    model.eval()
    total_loss = 0.0
    total_samples = 0
    all_preds = []
    all_labels = []
    

    with torch.no_grad():
        for batch in test_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            loss = model.calculate_loss(batch)
            bs = len(batch['rating'])
            total_loss += loss.item() * bs
            total_samples += bs

            preds = model.predict(batch)
            labels = batch['rating'] 

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_loss = total_loss / total_samples if total_samples > 0 else 0.0

    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    mae = mean_absolute_error(all_labels, all_preds)
    mse = mean_squared_error(all_labels, all_preds)
    rmse = np.sqrt(mse)
    r2 = r2_score(all_labels, all_preds)
    return avg_loss, mae, rmse, r2


def evaluate_testset(
    server_round: int,
    parameters: NDArrays,
    config: Dict[str, Scalar],
) -> Optional[Tuple[float, Dict[str, Scalar]]]:
    model = DMFFederated(
            num_users=len(global_user_id_map),
            num_items=len(global_movie_id_map),
            user_embedding_size=32,
            item_embedding_size=32,
            user_hidden_sizes=[64, 32],
            item_hidden_sizes=[64, 32],
            dropout=0.3,
            activation="leaky_relu",
            bn=False,
            init_method="norm"
            ).to(device)
    
    _, _, testloader = load_datasets(0, batch_size,  global_user_id_map, global_movie_id_map)
    set_parameters(model, parameters)  # Update model with the latest parameters
    loss, mae, rmse, r2 = evaluate_DMFRegressor(model, testloader, device=device)
    print(f"Server-side evaluation loss {loss}, mae {mae}, rmse {rmse}, r2 {r2}")
    return loss, {"mae": mae, "rmse": rmse, "r2": r2}

In [12]:
def fit_config(server_round: int):
    """Return training configuration dict for each round.

    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "server_round": server_round, 
        "local_epochs": 15 
    }
    return config


num_rounds = 10
def server_fn(context: Context) -> ServerAppComponents:
    # Create FedAvg strategy
    strategy = FedAvg(
        fraction_fit=1,
        fraction_evaluate=1,
        min_fit_clients=NUM_PARTITIONS,
        min_evaluate_clients=NUM_PARTITIONS,
        min_available_clients=NUM_PARTITIONS,
        initial_parameters=ndarrays_to_parameters(params),
        on_fit_config_fn=fit_config,  
        evaluate_fn=evaluate_testset
    )
    config = ServerConfig(num_rounds=num_rounds)
    return ServerAppComponents(strategy=strategy, config=config)

backend_config = {"client_resources": None}
if DEVICE.type == "cuda":
    backend_config = {"client_resources": {"num_gpus": 2, "num_cpus": 1}}


server_app = ServerApp(server_fn=server_fn)
server_app


### Simulation

In [13]:
########################################
# Simulation
########################################

import time

before_sim_start = time.time()

run_simulation(
    server_app=server_app,
    client_app=client_app,
    num_supernodes=NUM_PARTITIONS,
    backend_config=backend_config 
)

after_sim_start = time.time()
print(f"Simulation took {after_sim_start - before_sim_start} second to finish")

INFO :      Starting Flower ServerApp, config: num_rounds=10, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Evaluating initial global parameters


(142621, 3) (35656, 3) (98138, 3)


INFO :      initial parameters (loss, other metrics): 1.6509557969292865, {'mae': 3.5519116, 'rmse': 3.7049718, 'r2': -11.3582637099747}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)


Server-side evaluation loss 1.6509557969292865, mae 3.5519115924835205, rmse 3.7049717903137207, r2 -11.3582637099747
(ClientAppActor pid=2480272) (142621, 3) (35656, 3) (98138, 3)
(ClientAppActor pid=2480272) [Client 0] fit, config: {'server_round': 1, 'local_epochs': 15}
(ClientAppActor pid=2480272) Epoch 1/1, Training Loss: 1.0262
(ClientAppActor pid=2480272) (142620, 3) (35656, 3) (98138, 3)
(ClientAppActor pid=2480272) [Client 2] fit, config: {'server_round': 1, 'local_epochs': 15}
(ClientAppActor pid=2480272) Epoch 1/1, Training Loss: 1.0219
(ClientAppActor pid=2480272) (142620, 3) (35656, 3) (98138, 3)
(ClientAppActor pid=2480272) [Client 4] fit, config: {'server_round': 1, 'local_epochs': 15}
(ClientAppActor pid=2480272) Epoch 1/1, Training Loss: 1.0210
(ClientAppActor pid=2480272) (142621, 3) (35656, 3) (98138, 3)
(ClientAppActor pid=2480272) [Client 1] fit, config: {'server_round': 1, 'local_epochs': 15}
(ClientAppActor pid=2480272) Epoch 1/1, Training Loss: 1.0218
(ClientApp

INFO :      aggregate_fit: received 5 results and 0 failures


...

(ClientAppActor pid=2480272) Epoch 1/1, Training Loss: 0.3438
(142621, 3) (35656, 3) (98138, 3)


INFO :      fit progress: (10, 0.2354739606866347, {'mae': 0.6783642, 'rmse': 0.8923807, 'r2': 0.28305148623620524}, 509.3466588156298)
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


Server-side evaluation loss 0.2354739606866347, mae 0.6783642172813416, rmse 0.8923807144165039, r2 0.28305148623620524
(ClientAppActor pid=2480272) (142621, 3) (35656, 3) (98138, 3)
(ClientAppActor pid=2480272) [Client 1] evaluate, config: {}
(ClientAppActor pid=2480272) Evaluation Loss: 0.2344
(ClientAppActor pid=2480272) (142620, 3) (35656, 3) (98138, 3)
(ClientAppActor pid=2480272) [Client 2] evaluate, config: {}
(ClientAppActor pid=2480272) Evaluation Loss: 0.2365
(ClientAppActor pid=2480272) (142621, 3) (35656, 3) (98138, 3)
(ClientAppActor pid=2480272) [Client 0] evaluate, config: {}
(ClientAppActor pid=2480272) Evaluation Loss: 0.2350
(ClientAppActor pid=2480272) (142620, 3) (35656, 3) (98138, 3)
(ClientAppActor pid=2480272) [Client 3] evaluate, config: {}
(ClientAppActor pid=2480272) Evaluation Loss: 0.2358
(ClientAppActor pid=2480272) (142620, 3) (35656, 3) (98138, 3)
(ClientAppActor pid=2480272) [Client 4] evaluate, config: {}


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 10 round(s) in 527.51s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.3252923802393874
INFO :      		round 2: 0.2653364297036823
INFO :      		round 3: 0.24994945465408422
INFO :      		round 4: 0.24340145983034273
INFO :      		round 5: 0.23976447070182536
INFO :      		round 6: 0.23691898496436622
INFO :      		round 7: 0.23535550273812775
INFO :      		round 8: 0.23433053093304781
INFO :      		round 9: 0.23534000892729565
INFO :      		round 10: 0.23531724695122133
INFO :      	History (loss, centralized):
INFO :      		round 0: 1.6509557969292865
INFO :      		round 1: 0.3247440861144296
INFO :      		round 2: 0.2657425680093519
INFO :      		round 3: 0.2501269211374372
INFO :      		round 4: 0.24351011577288112
INFO :      		round 5: 0.2397434278478704
INFO :      		round 6: 0.23693703850767125
INFO :      		round 7: 0.23553123856894692

(ClientAppActor pid=2480272) Evaluation Loss: 0.2350
Simulation took 539.3246395587921 second to finish
